
# Linear Regression: Closed-Form Solution


There are two very different solution approaches for the Linear Regression problem.
    - The “closed-form” solution approach.
    - Iterative optimization approach known as Gradient Descent (GD).
    


In this notebook we investigate the implementation of the **closed-form** solution for the Linear Regression problem.

The “closed-form” solution directly computes the model parameters (weights) that best fit the model to the training set. I.e., the model parameters that minimize the cost function over the training set.


### Three Scenarios

Depending on the type of the data matrix X, following three scenarios may arise in the closed-form solution approach:
    - Scenario 1: Data Matrix X is Square
    - Scenario 2: Data Matrix X is Non-Square (No Collinearity)
    - Scenario 3: Data Matrix X is Non-Square (Collinearity)

For the last two scenarios we need to use a special closed-form technique known as the **Ordinary Least Squares (OLS)** method.


### Summary of the Solution Approach for three Scenarios

- Scenario 1: Data Matrix X is Square

        We use a simple Linear Algebra trick (iverting the data matrix) to derive the closed-form solution.


- Scenario 2: Data Matrix X is Non-Square (No Collinearity)
        
        We implement a special technique called the Ordinary Least Squares (OLS) method to derive the closed-form solution.


- Scenario 3: Data Matrix X is Non-Square (Collinearity)

        We implementat a regularized (penalized) least square method or Ridge regression method to derive the closed-form solution.



#### <font color=red>Note: </font>
We implement the OLS method (for scenario 2 & 3) using only the **NumPy** library. It helps to understand the structure of the OLS solution. There is also a **Scikit-Learn implementation** of the OLS method for the Linear Regression problem. We will use that solution in a different notebook. This notebook does not use any Scikit-Learn solutions.  



In [1]:
import numpy as np
from numpy.linalg import inv, det, matrix_rank
from sklearn.metrics import mean_squared_error, r2_score

# Some Useful Linear Algebra Techniques Using NumPy

Before we perform the investigation, we review some basic Linear Algebra techniques for:
- Representation of a feature/weight vector using a numpy array
- Computing dot product between the feature vectors and the weight vectors



## Creating the Target by Using Feature and the Weight

The prediction or target vector ("y") is created by taking the dot product between the feature vector (denoted by lower case "x") and the weight vector (denoted by "w").

We will discuss two scenarios for computing the dot product:
1. A single feature vector (d-dimensional)
2. Multiple feature vectors (d-dimensional)

###### Note that a data matrix (denoted by upper case "X") is constructed by using multiple d-dimensiional feature vectors.


## Dot Product between a Single Feature Vector ($\vec x$) and the Weight Vector ($\vec w$)

We compute the dot product between the feature vector $\vec x$ and the weight vector $\vec w$ in one of the following two ways:

1. Method 1: Create the two vectors as **column vectors**, then take the transpose of either $\vec x$ or $\vec w$. Compute the dot product between them.
2. Method 2: Create the two vectors as **row vectors**. Then, compute the dot product. Don't need to compute transpose.


#### <font color=red>Method 1 is often used.</font>


### Method 1: Create the two vectors as column vectors, then take the transpose of either x or w.  Compute the dot product between them.

In [9]:

'''
Define a single sample "x" with 3 features as a column vector
Define a 3D weight vector "w" as a column vector

'''


# sample x with 3D feature is defined as a column vector
x = np.array([1, 5, 10]).reshape(-1, 1)

# 3D weight vector is defined as a column vector 
w = np.array([5, 6, 1]).reshape(-1, 1)


print("x:\n", x)
print("w:\n", w)


print("\nTranspose of w:", w.T)
print("Transpose of x:", x.T)


print("\nDot Product of x and w:")


y1 = x.T.dot(w)
print("y1 (x^T.w):", y1)


y2 = w.T.dot(x)
print("y2 (w^T.x):", y2)

x:
 [[ 1]
 [ 5]
 [10]]
w:
 [[5]
 [6]
 [1]]

Transpose of w: [[5 6 1]]
Transpose of x: [[ 1  5 10]]

Dot Product of x and w:
y1 (x^T.w): [[45]]
y2 (w^T.x): [[45]]


### Method 2: Define the two vectors as row vectors. Then, compute the dot product. 

#### <font color=red>Don't need to compute transpose.</font>

In [10]:
'''
Define a single sample "x" with 3 features as a row vector
Define a 3D weight vector "w" as a row vector
'''

w = np.array([5, 6, 1])
x = np.array([1, 5, 10])


print("w:", w)
print("x:", x)


print("\nDot Product of x and w:")

y1 = w.dot(x)
print("\nh1 (w.x):", y1)


y2 = x.dot(w)
print("\nh2 (x.w):", y2)


w: [5 6 1]
x: [ 1  5 10]

Dot Product of x and w:

h1 (w.x): 45

h2 (x.w): 45


## Dot Product Between Data Matrix X ($N$ Features) and the Weight Vector ($\vec w$)

A data matrix is denoted by upper case "X". 

It is created by transposing the "N" feature vectors (d-dimensional colmun vectors) and adding them as rows (number of rows = N).

Each row of X represents a data point (transposed feature vector).

In [11]:
# Define the 3D weight vector as a column vector
w = np.array([5, 6, 1]).reshape(-1, 1)


# The data matrix "X" is created by transposing the "N" feature vectors (d-dimensional colmun vectors) 
#   and adding them as rows (number of rows = N)
# The dimension of this matrix is N x d
X = np.array([[1, 5, 10],
              [1, 4, 9],
              [1, 3, 13],
              [1, 2, 6]
             ])

print("w:\n", w)
print("\nX:\n", X)
print("\nNote: Each row of X represents a transposed feature vector.")

# The target vector y is created by taking the dot product between X and w.
# Note: we don't have to apply transpose as we dis in case a single feature
y = X.dot(w)
print("\ny:\n", y)


w:
 [[5]
 [6]
 [1]]

X:
 [[ 1  5 10]
 [ 1  4  9]
 [ 1  3 13]
 [ 1  2  6]]

Note: Each row of X represents a transposed feature vector.

y:
 [[45]
 [38]
 [36]
 [23]]


# Solving the Linear Regression Problem

The Linear Regression problem is modeled by a system of linear equations or linear system.
- $\vec X. \vec w = \vec y$

The goal is to learn the unknown weight vector ($\vec w$) of a linear system.

Depending on the dimension of the data Matrix X, the following two scenarios arise:
- Data Matrix X is Square
- Data Matrix X is Non-Square

For square X, the solution is trivial.

However, when X is non-square, we need to use a special method called the **Ordinary Least Squares (OLS)**. 


Below we solve the linear system for the above two scenarios by finding the unknown weight vector $\vec w$, then using it, we predict the target vector. Finally, we evaluate the prediction performance by using the following two evaluation metrics. 


## Evaluation Metrics

We will use two evaluation metrics.

- Mean Squared Error (MSE)
- Coefficient of Determination ($R^2$ or $r^2$)


### Note on $R^2$:
R-squared is a statistical measure of **how close the data are to the fitted regression line**. 

R-squared metric measures the proportion of the variance in the dependent variable that is predictable from the independent variable(s).

$R^2 = \frac{Explained Variation}{Total Variation}$

R-squared is always between 0 and 100%:

- 0% indicates that the model explains none of the variability of the response data around its mean.
- 100% indicates that the model explains all the variability of the response data around its mean.


#### <font color=red>In general, the higher the R-squared value, the better the model fits your data.</font>


#### Compute $R^2$ using the sklearn:

- The "r2_score" function from sklearn.metrics

#### Compute MSE using the sklearn:

- The "mean_squared_error" function from sklearn.metrics


## Data Matrix X is Square

When the data matrix X is square, the OLS solution is given by:

 $\vec w = \vec X^{-1} . \vec y$

In [12]:
# Linear Regression: Square Data Matrix


# Create a data matrix that becomes a square matrix after adding the bias term 
X = np.array([[13, 22, 35],
              [17, 13, 9],
              [7, 3, 12],
              [1, 31, 1]])



# Add a bias term with the feature vectors
X_bias = np.c_[np.ones((X.shape[0],1)),X]


# Create the target vector
y = np.array([22, 34, 45, 67])


# OLS solution
w = np.linalg.inv(X_bias).dot(y)

print("\nX:\n", X)
print("\ny:\n", y)

print("\nWeight Vector: ", w)



print("\n----------------------------- Model Evaluation -----------------------------")



# Make prediction 
y_predicted = X_bias.dot(w)


print("Mean squared error: %.2f"
      % mean_squared_error(y, y_predicted))


# Explained variance score: 1 is perfect prediction
print("Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" % r2_score(y, y_predicted))



X:
 [[13 22 35]
 [17 13  9]
 [ 7  3 12]
 [ 1 31  1]]

y:
 [22 34 45 67]

Weight Vector:  [63.96483756 -1.49554279  0.1703645  -0.75059429]

----------------------------- Model Evaluation -----------------------------
Mean squared error: 0.00
Coefficient of determination r^2 variance score [1 is perfect prediction]: 1.00


## Data Matrix X is Non-Square

If the data matrix X is not square, then we need to solve the **Normal equation**, instead of the $\vec X. \vec w = \vec y$ linear system of equations.

The solution to the Normal equation is known as the Ordinary Least Squares (OLS) solution.

Below we implement the OLS method for Linear Regression.

## Ordinary Least Squares (OLS) Method


The Ordinary Least Squares (OLS) is a type of linear least squares method for estimating the unknown parameters ($\vec w$) in a linear regression model. We can find the optimal weight vectors of linear regression by using the OLS method.

The OLS method is used to solve a linear system of equations defined by the **Normal Equation.**

##### Normal Equation:  $X^TXw = X^TY$

The OLS solution to the Normal equation is: $\hat w_{OLS} = (X^TX)^{-1}X^TY$

Therefore, given the non-square data matrix X and the true target vector Y, we can find $\hat w_{OLS}$.


### Note: 
The OLS solution **<font color=red>doesn't work</font>** if the columns of data matrix X has colinearity. In this case, we cannot compute $(X^TX)^{-1}$ as it becomes **singular**.

Thus, we consider two cases for non-square data matrix X:
- Non-Square X: No Collinearity in the Columns
- Non-Square X: Collinearity in the Columns

## Non-Square X: No Collinearity in the Columns

We create a data matrix X that doesn't have collinearity in the columns.

In [13]:
# OLS Method for Linear Regression


# Create a data matrix that doesn't have collinearity in the columns
X = np.array([[13, 22, 35],
              [17, 13, 9]])

# Create the target vector
y = np.array([22, 34])


print("\nX:\n", X)
print("\ny:\n", y)


print("\nRank of X: ", matrix_rank(X))

# Add a bias term with the feature vectors
X_bias = np.c_[np.ones((X.shape[0],1)),X]
print("\nRank of X_bias: ", matrix_rank(X_bias))


# The determinant should be non-zero
print("\nDeterminant of (X_bias^T.X_bias): ", det(X_bias.T.dot(X_bias)))


# Computes the product of the transpose of X_bias with itself
z = X_bias.T.dot(X_bias) 

print("\nRank of z: ", matrix_rank(z))


# Closed form (OLS) solution for weight vector w 
w = np.linalg.inv(z).dot(X_bias.T).dot(y)




print("\nWeight vector:\n",w)





print("\n----------------------------- Model Evaluation -----------------------------")



# Make prediction (you should use test data)
y_predicted = X_bias.dot(w)


print("Mean squared error: %.2f"
      % mean_squared_error(y, y_predicted))


# Explained variance score: 1 is perfect prediction
print("Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" % r2_score(y, y_predicted))



X:
 [[13 22 35]
 [17 13  9]]

y:
 [22 34]

Rank of X:  2

Rank of X_bias:  2

Determinant of (X_bias^T.X_bias):  -4.633503900004057e-24

Rank of z:  2

Weight vector:
 [-24.625        4.51171875   1.12695312  -0.84179688]

----------------------------- Model Evaluation -----------------------------
Mean squared error: 343.29
Coefficient of determination r^2 variance score [1 is perfect prediction]: -8.54


## Non-Square X: Collinearity in the Columns

Note that the OLS solution **<font color=red>doesn't work</font>** if the columns of data matrix X has collinearity. In this case, we cannot compute $(X^TX)^{-1}$ as it becomes **singular**.

Below we will create a data matrix X that has collinearity in its columns. We will see that $(X^TX)^{-1}$ suffers from **singularity**.

To verify the singularity of $(X^TX)^{-1}$, we compute its eigenvalues. If all eigenvalues are not positive, then this matrx will **not be positive definite**, consequently singular.

We will see that $(X^TX)^{-1}$ is not positive definitie (i.e., not all of its eigenvalues are positive). As a consequence, the matrix is not invertible.

After investigating possible singularity in $(X^TX)^{-1}$, we implement a technique for fixing the singularity problem of $(X^TX)$.

In [14]:
# Create a matrix "X" N x d in which N >> d.
# Some columns of X are linearly dependent.
# Then, compute "X^T.X" and see if it's invertible.


# Create a non-square matrix & make a column linearly dependent on another column
X = np.array([[1, 2, 3, 4],
              [7, 14, 11, 12]])


#This matrix doesn't have singularity (you can use this for experimentation)
# X = np.array([[1, 13, 22, 35],
#               [1, 17, 13, 1]])



print("\nX (N x d):\n")
print(X)

print("\nRank of X: ", matrix_rank(X))


# Compute X^T.X
Z = X.T.dot(X)
print("\nZ (= X^T.X):\n")
print(Z)
print("\nRank of Z: ", matrix_rank(Z))



print("\nDeterminant of Z (X^T.X): ", det(Z))

# Find the eigenvalues of X^T.X
# For this particular example (X), not all eigenvalues will be positive
# Hence, X^T.X is not positive definite & is NOT invertible

eigenvalues, eigenvectors = np.linalg.eig(Z)

print("\nEigenvalues:\n")
print(eigenvalues)




# Count the number of zero/negative eigenvalues
noOfZeroEigenvalues = 0
for i in range(len(eigenvalues)):
    if (eigenvalues[i] > 0):
        noOfZeroEigenvalues = 0
    else:
        noOfZeroEigenvalues += 1
        
print("\nNumber of zero/negative eigenvalues: %d" % noOfZeroEigenvalues)


# Invert X^T.X
Z_inv = np.linalg.inv(Z)
print("\nInverse of X^T.X (d x d):\n")
print(Z_inv)



X (N x d):

[[ 1  2  3  4]
 [ 7 14 11 12]]

Rank of X:  2

Z (= X^T.X):

[[ 50 100  80  88]
 [100 200 160 176]
 [ 80 160 130 144]
 [ 88 176 144 160]]

Rank of Z:  2

Determinant of Z (X^T.X):  0.0

Eigenvalues:

[ 5.36563313e+02  3.43668670e+00 -2.40578862e-14 -1.15269744e-14]

Number of zero/negative eigenvalues: 2


LinAlgError: Singular matrix

## Fixing the Singularity Problem of $(X^TX)^{-1}$

We have observed the $(X^TX)^{-1}$ in the previous example was **NOT positive definitie**, and hence was singular.

One intuitive trick to solve the singularity problem of $(X^TX)^{-1}$ is to add some positive small numbers (known as penalty) on its diagonal. It will make its eigenvalues positive.

Later we will see that this is a **Bayesian** solution to the non-invertibility problem of $(X^TX)^{-1}$d.

This Bayesian solution is known as **<font color=blue> Ridge Regression. </font>** or regularized (penalized) least square method.

In [15]:
# Create a non-square matrix & make a column linearly dependent on another column
X = np.array([[1, 2, 3, 4],
              [7, 14, 11, 12]])


print("X (N x d):")
print(X)

print("\nRank of X: ", matrix_rank(X))

# Compute X^T.X
Z = np.transpose(X).dot(X)
print("\nZ (= X^T.X):")
print(Z)


print("\nDeterminant of Z (X^T.X): ", det(Z))



# Find the eigenvalues of X^T.X
# For this particular example (X), some eigenvalues may not be positive
# Hence, X^T.X is not positive definite & is NOT invertible

eigenValues, eigenVectors = np.linalg.eig(Z)

print("\nEigenvalues of Z:")
print(eigenValues)


# Count the number of zero/negative eigenvalues
noOfZeroEigenvalues = 0
for i in range(len(eigenvalues)):
    if (eigenvalues[i] > 0):
        noOfZeroEigenvalues = 0
    else:
        noOfZeroEigenvalues += 1
        
print("\nNumber of zero/negative eigenvalues: %d" % noOfZeroEigenvalues)


# Invert X^T.X
# Z_inv = np.linalg.inv(Z)
# print("\nInverse of X^T.X (d x d):\n")
# print(Z_inv)


print("\n-------- Fixing the Singularity of (X^T)X ------------")


# Create a digonal matrix with small positive numbers on the diagonal
diagonal = np.array([[0.001, 0, 0, 0],
                     [0, 0.001, 0, 0],
                     [0, 0, 0.001, 0],
                     [0, 0, 0, 0.001]])


print("\nDiagonal Matrix:")
print(diagonal)


# Add the diogonal matrix with Z
print("\nZ + Diagonal Matrix:")
Z = Z + diagonal

print(Z)

eigenvalues, eigenvectors = np.linalg.eig(Z)

print("\nEigenvalues of (Z + Diagonal):")
print(eigenvalues)


# Count the number of zero/negative eigenvalues
noOfZeroEigenvalues = 0
for i in range(len(eigenvalues)):
    if (eigenvalues[i] > 0):
        noOfZeroEigenvalues = 0
    else:
        noOfZeroEigenvalues += 1
        
print("\nNumber of zero/negative eigenvalues: %d" % noOfZeroEigenvalues)

print("\nDeterminant of Z (X^T.X) after adding the diagonal matrix: ", det(Z))

# Invert X^T.X
Z_inv = np.linalg.inv(Z)
print("\nInverse of X^T.X:")
print(Z_inv)


X (N x d):
[[ 1  2  3  4]
 [ 7 14 11 12]]

Rank of X:  2

Z (= X^T.X):
[[ 50 100  80  88]
 [100 200 160 176]
 [ 80 160 130 144]
 [ 88 176 144 160]]

Determinant of Z (X^T.X):  0.0

Eigenvalues of Z:
[ 5.36563313e+02  3.43668670e+00 -2.40578862e-14 -1.15269744e-14]

Number of zero/negative eigenvalues: 2

-------- Fixing the Singularity of (X^T)X ------------

Diagonal Matrix:
[[0.001 0.    0.    0.   ]
 [0.    0.001 0.    0.   ]
 [0.    0.    0.001 0.   ]
 [0.    0.    0.    0.001]]

Z + Diagonal Matrix:
[[ 50.001 100.     80.     88.   ]
 [100.    200.001 160.    176.   ]
 [ 80.    160.    130.001 144.   ]
 [ 88.    176.    144.    160.001]]

Eigenvalues of (Z + Diagonal):
[5.36564313e+02+0.0000000e+00j 3.43768670e+00+0.0000000e+00j
 1.00000000e-03+9.3634816e-15j 1.00000000e-03-9.3634816e-15j]

Number of zero/negative eigenvalues: 0

Determinant of Z (X^T.X) after adding the diagonal matrix:  0.0018445400010404084

Inverse of X^T.X:
[[ 806.97084379 -386.05831243  -69.43736646   43.323

## Applying OLS Method on Data Matrix With Collinearity in Columns

Now that we know how to fix the singularity problem of the $(X^T)X$ matrix, we can apply the OLS method on a data matrix that has collinearity in its columns.

Note that the singularity problem can be solved by adding small positive numbers on the diagonal of the $(X^T)X$ matrix.

We will see later that this technique is not ad-hoc. It is a natural consequence of applying the Bayesian technique on the OLS. The Bayesian technique is known as the **Ridge Regression**.

In [16]:

# data matrix X that creates singularity for (X^T)X 

X = np.array([[1, 2, 35],
              [7, 14, 9]])

y = np.array([22, 34])

# Add a bias term with the feature vectors
X_bias = np.c_[np.ones((X.shape[0],1)),X]


# The determinant should be zero as X_bias^T.X_bias is singular
print("\nDeterminant of (X_bias^T.X_bias): ", det(X_bias.T.dot(X_bias)))



# Computes the product of the transpose of X_bias with itself
z = X_bias.T.dot(X_bias) 


print("\n-------- Fixing the Singularity of (X_bias^T)X_bias ------------")

# Create a diagonal matrix that has the dimension of z
diagonal = np.eye(z.shape[0], dtype=float)

# Add small non-zero numbers on the diagonal
diagonal = diagonal * 0.001
print("The diagonal matrix:\n", diagonal)

# Closed form solution for weight vector w using Ridge Regregression
w = np.linalg.inv(z + diagonal).dot(X_bias.T).dot(y)

print("\nThe weight vector:\n",w)





print("\n----------------------------- Model Evaluation -----------------------------")



# Make prediction (you should use test data)
y_predicted = X_bias.dot(w)


print("Mean squared error: %.2f"
      % mean_squared_error(y, y_predicted))


# Explained variance score: 1 is perfect prediction
print("Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" % r2_score(y, y_predicted))



Determinant of (X_bias^T.X_bias):  0.0

-------- Fixing the Singularity of (X_bias^T)X_bias ------------
The diagonal matrix:
 [[0.001 0.    0.    0.   ]
 [0.    0.001 0.    0.   ]
 [0.    0.    0.001 0.   ]
 [0.    0.    0.    0.001]]

The weight vector:
 [0.10522057 0.83835579 1.67671159 0.5058005 ]

----------------------------- Model Evaluation -----------------------------
Mean squared error: 0.00
Coefficient of determination r^2 variance score [1 is perfect prediction]: 1.00
